In [1]:
import numpy as np
import pandas as pd

from statistics import median

import math
from time import time
from sklearn.preprocessing import MinMaxScaler

In [2]:
train_data = pd.read_csv('../train.data',delimiter=',')
matrix = np.zeros((943,1682),dtype=int)
for row in train_data.to_numpy():
    matrix[row[0]-1,row[1]-1] = row[2]
test = pd.read_csv('../test.data').to_numpy()

In [3]:
Rmin = 1
Rmax = 5
Rmid = int((5+1)/2)

def med(arr1,arr2):
    midup = []
    middown = []
    midup = arr1[arr1[:]>=3]
    np.append(midup,arr2[arr2[:]>=3])
    middown = arr1[arr1[:]<3]
    np.append(middown,arr2[arr2[:]<3])
    return median(midup),median(middown)

def pip(r1,r2,midplus,midminus,col_no,avg):
    if (r1>=Rmid and r2>=Rmid) or (r1<Rmid and r2<Rmid):
        prox = (((abs(r1-r2))-(midminus+midplus)/2)/(Rmax-Rmin))**2
        imp = math.pow(math.e,-(1/(abs(r1-Rmid)+1)*(abs(r2-Rmid)+1)))
        pop = math.log10((2+(((r1+r2)/2)-avg[col_no])**2))
    else:
        dst = abs(r1-r2)
        if dst > Rmid:
            delta = 0.75
        elif dst == Rmid:
            delta = 0.5
        else:
            delta = 0.25
        prox = delta*(((Rmax-Rmin)/delta)**2)
        imp = 1/(abs(r1-Rmid)+1)*(abs(r2-Rmid)+1)
        pop = 0.3010
    return prox*imp*pop



def MPIP(u1,u2,item,avg,midplus,midminus):
    prox = pip(matrix[u1,item],matrix[u2,item],midplus,midminus,item,avg)
    return prox

def sim(u1,u2):
    sum=0
    avg = matrix.mean(axis=0)
    midplus,midminus = med(matrix[u1],matrix[u2])
    for i in range(0,1682):
        sum+=MPIP(u1,u2,i,avg,midplus,midminus)
    return sum

In [4]:
sim(1,1)

52.31352951128804

In [6]:
Mpip = np.ones((943,943))
for i in range(943):
        for j in range(943):
            if i==j:
                Mpip[i,j]=654.298
                break
            Mpip[i,j] = sim(i,j)
            Mpip[j,i] = Mpip[i,j]

In [7]:
Mpip = pd.DataFrame(Mpip)
Mpip.to_csv('../mpip.csv',sep=',',index=False)

In [8]:
Mpip

,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,654.298000,1367.282640,1134.187476,855.284954,1979.826293,1948.284571,4656.615437,1080.937841,849.451715,2033.175920,...,2154.546522,967.960231,2091.961954,877.827484,1811.662162,1127.370093,1512.998211,786.032465,1418.019817,1680.069207
1,1367.282640,654.298000,670.780583,334.922534,2230.510088,2347.420062,5324.265283,807.603260,379.799684,2122.614065,...,2147.293901,521.219497,1761.305264,381.006565,1362.188135,722.289754,1440.965912,333.331152,1105.263544,1828.002364
2,1134.187476,670.780583,654.298000,257.131854,2214.521308,2383.707107,5292.574200,762.888575,343.255474,2153.724521,...,2201.413571,499.129653,2098.133114,536.339630,1511.900383,665.550960,1477.570279,340.012521,1054.823372,1769.272781
3,855.284954,334.922534,257.131854,654.298000,2210.839496,2412.158889,5351.895386,669.557740,332.616789,2151.470804,...,2210.639139,493.187994,1982.368657,494.490459,1433.736064,670.121295,1509.453395,300.575367,1000.137403,1779.683342
4,1979.826293,2230.510088,2214.521308,2210.839496,654.298000,2247.325029,4821.692469,846.624213,532.355535,2088.910158,...,2055.674311,643.887312,2224.005979,685.396698,1640.705539,882.745465,1412.593836,507.021638,1102.978931,1742.384152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,1127.370093,722.289754,665.550960,670.121295,882.745465,1023.310980,1585.168195,735.294581,660.059637,1135.288343,...,911.037522,555.207445,778.714798,637.011828,586.908473,654.298000,1595.423811,389.062399,1103.711301,1760.399879
939,1512.998211,1440.965912,1477.570279,1509.453395,1412.593836,1311.928001,1846.653436,1370.043467,1461.194147,1406.667124,...,1412.150016,1486.840344,1594.629602,1408.818714,1423.562564,1595.423811,654.298000,426.757852,1040.313491,1681.656645
940,786.032465,333.331152,340.012521,300.575367,507.021638,657.510140,1309.521810,339.720854,322.954548,716.718080,...,646.825241,314.396511,474.763773,286.578382,278.501390,389.062399,426.757852,654.298000,1030.052933,1714.840697
941,1418.019817,1105.263544,1054.823372,1000.137403,1102.978931,1160.522722,1690.939247,981.871720,1061.481099,1295.964617,...,1159.981624,1088.186748,1225.209161,1055.760713,1083.219654,1103.711301,1040.313491,1030.052933,654.298000,1739.093557
